In [1]:
import crs_toolkit
from crs_toolkit import ChatgptAgent, ChatgptAgentConfig
from crs_toolkit import ChatgptGen, ChatgptRec
from dotenv import load_dotenv
import os

/Users/jun/Desktop/25_1/RecSys_and_LLM/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from crs_toolkit import LlamaGen, LlamaRec, LlamaAgent, LlamaAgentConfig

In [3]:
load_dotenv()

True

In [4]:
os.environ["HF_AUTH_TOKEN"] = os.getenv("HF_AUTH_TOKEN")

In [5]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
gpt_rec = ChatgptRec.from_pretrained('PNUDI/ChatGPT-rec')
gpt_gen = ChatgptGen.from_pretrained('PNUDI/ChatGPT-gen')
chat_gpt_config = ChatgptAgentConfig()
pipeline_gpt = ChatgptAgent(
    config=chat_gpt_config,
    rec_module=gpt_rec,
    gen_module=gpt_gen
)

Config: ChatgptAgentConfig {
  "WEIGHT_DIMENSIONS": {},
  "_attn_implementation_autoset": true,
  "answer_type": "movie",
  "model_name": "gpt-4o",
  "prompt": "",
  "rec_pattern": "<movie>",
  "resp_prompt": "System:",
  "transformers_version": "4.49.0"
}



In [6]:
llama_rec = LlamaRec.from_pretrained("PNUDI/llama-rec")

In [7]:
llama_gen = LlamaGen.from_pretrained("PNUDI/llama-gen")

In [8]:
llama_config = LlamaAgentConfig()
llama_pipeline = LlamaAgent(
    config=llama_config,
    rec_module=llama_rec,
    gen_module=llama_gen
)

Config: LlamaAgentConfig {
  "WEIGHT_DIMENSIONS": {},
  "_attn_implementation_autoset": true,
  "answer_type": "movie",
  "model_name": "meta-llama/Llama-3.2-1B-Instruct",
  "prompt": "",
  "transformers_version": "4.49.0"
}



In [9]:
def set_pipeline():
    return llama_pipeline

In [10]:
def chat_with_system(pipeline):
    context = []
    print("대화를 시작합니다. 종료하려면 'quit'를 입력하세요.")

    while True:
        user_input = input("User: ")
        if user_input.lower() == 'quit':
            break

        # 사용자 입력을 컨텍스트에 추가
        context.append(f"User: {user_input}")

        # 전체 컨텍스트를 <sep>로 연결
        full_context = "<sep>".join(context)

        # 응답 생성
        response = pipeline.response(full_context)
        print("System:", response)

        # 시스템 응답을 컨텍스트에 추가
        context.append(f"System: {response}")

In [11]:
dialog = [
    "User: Hello!",
    "System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?",
    "User: Yes, i've seen that movie. And i'm Looking for other movies in the horor category."
]

In [12]:
def build_llama_prompt(dialog, user_input=""):
    history = ""
    last_user_input = user_input
    for turn in dialog[:-1]:
        history += turn + "\n"
    if dialog[-1].startswith("User:"):
        last_user_input = dialog[-1].replace("User:", "").strip()
    else:
        raise ValueError("Last dialog entry must be from the user.")
    prompt = (
        "<|begin_of_text|>\n"
        "<|start_header_id|>system\n" + history.strip() + "\n<|end_header_id|>\n"
        "<|start_header_id|>user\n" + last_user_input + "\n<|end_header_id|>\n"
        "<|end_of_text|>"
    )
    return prompt

In [13]:
# This is for ChatGPT
def test_single_turn(pipeline, dialog):
    # 대화 컨텍스트 설정
    context = "<sep>".join(dialog)
    # print(context)
    answer = pipeline.response(context)
    print(answer)

test_single_turn(pipeline_gpt, dialog)

[ChatgptAgent] user query: User: Hello!<sep>System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?<sep>User: Yes, i've seen that movie. And i'm Looking for other movies in the horor category.
[ChatgptGen] raw_input: User: Hello!<sep>System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?<sep>User: Yes, i've seen that movie. And i'm Looking for other movies in the horor category.
[ChatgptGen] tokenized messages: [{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hello, I have some movie ideas for you. Have you watched the movie Interstella?'}, {'role': 'user', 'content': "Yes, i've seen that movie. And i'm Looking for other movies in the horor category."}, {'content': ' You are generating templates for a program to fill. In the template, each movie is replaced with a <movie> mask. For example, when you intend to say: "Inception is a good movie". You should instead sa

[ChatgptGen] raw response from model: Great! If you're interested in horror movies, you might enjoy watching <movie> or <movie>. Both are well-regarded in the genre and offer some great scares. Do you have any specific type of horror you're interested in, like psychological, supernatural, or slasher?
[ChatgptGen] final return (string): System: Great! If you're interested in horror movies, you might enjoy watching <movie> or <movie>. Both are well-regarded in the genre and offer some great scares. Do you have any specific type of horror you're interested in, like psychological, supernatural, or slasher?
[ChatgptAgent] gen_output: System: Great! If you're interested in horror movies, you might enjoy watching <movie> or <movie>. Both are well-regarded in the genre and offer some great scares. Do you have any specific type of horror you're interested in, like psychological, supernatural, or slasher?
[ChatgptAgent] rec input: System: Great! If you're interested in horror movies, you might e

In [14]:
pipeline = set_pipeline()
prompt = build_llama_prompt(dialog)
print(prompt)

<|begin_of_text|>
<|start_header_id|>system
User: Hello!
System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?
<|end_header_id|>
<|start_header_id|>user
Yes, i've seen that movie. And i'm Looking for other movies in the horor category.
<|end_header_id|>
<|end_of_text|>


In [15]:
response = pipeline.response(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[LlamaAgent] user query: <|begin_of_text|>
<|start_header_id|>system
User: Hello!
System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?
<|end_header_id|>
<|start_header_id|>user
Yes, i've seen that movie. And i'm Looking for other movies in the horor category.
<|end_header_id|>
<|end_of_text|>
[LlamaGen] raw_input: <|begin_of_text|>
<|start_header_id|>system
User: Hello!
System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?
<|end_header_id|>
<|start_header_id|>user
Yes, i've seen that movie. And i'm Looking for other movies in the horor category.
<|end_header_id|>
<|end_of_text|>
[LlamaGen] prompt with system text: <|begin_of_text|>
<|start_header_id|>system
User: Hello!
System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?
<|end_header_id|>
<|start_header_id|>user
Yes, i've seen that movie. And i'm Looking for other movies in the horo

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.




system
User: Hello!
System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?

user
Yes, i've seen that movie. And i'm Looking for other movies in the horor category.

You are a helpful assistant. Generate concise sentence templates about movies. In the template, each movie name must be replaced with the <movie> mask.

Do not repeat the same instruction or list multiple times.

Example:
- "Inception is a good movie." → "<movie> is a good movie."

Return only 3 diverse templates using the <movie> mask. Avoid repeating similar sentence structures.

[LlamaGen] decoded_text: 
system
User: Hello!
System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?

user
Yes, i've seen that movie. And i'm Looking for other movies in the horor category.

You are a helpful assistant. Generate concise sentence templates about movies. In the template, each movie name must be replaced with the <movie> mask.

Do n

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[LlamaRec] generated_text: System: m
User: Hello!
System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?

user
Yes, i've seen that movie. And i'm Looking for other movies in the horor category.

You are a helpful assistant. Generate concise sentence templates about movies. In the template, each movie name must be replaced with the <movie> mask.

Do not repeat the same instruction or list multiple times.

Example:
- "Inception is a good movie." → "<movie> is a good movie."

Return only 3 diverse templates using the <movie> mask. Avoid repeating similar sentence structures. 
- "The Matrix is a classic movie." → "<movie> is a classic movie."
- "The Shining is a horror movie." → "<movie> is a horror movie."
- "The Silence of the Lambs is a psychological thriller." → "<movie> is a psychological thriller."

- "Inception is a mind-bending movie." → "<movie> is a mind-bending movie."
- "The Sixth Sense is a supernatural movie." → "<movie> is a 

In [16]:
response

'Final Answer:\nm\nUser: Hello!\n Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?\n\nuser\nYes, i\'ve seen that movie. And i\'m Looking for other movies in the horor category.\n\nYou are a helpful assistant. Generate concise sentence templates about movies. In the template, each movie name must be replaced with the <movie> mask.\n\nDo not repeat the same instruction or list multiple times.\n\nExample:\n- "Inception is a good movie." → "<movie> is a good movie."\n\nReturn only 3 diverse templates using the <movie> mask. Avoid repeating similar sentence structures. \n1. "<movie> is a cult classic."\n2. "<movie> is a thrilling adventure."\n3. "<movie> is a dark and mysterious horror story."'

In [17]:
def continue_conversation(dialog, user_input, pipeline):
    # 대화에 새로운 user 발화 추가
    dialog.append(f"User: {user_input}")

    # 프롬프트 생성
    prompt = build_llama_prompt(dialog)
    # 토크나이즈 및 모델 추론
    
    # 응답만 추출
    response = pipeline.response(prompt)

    # 대화 히스토리에 system 응답 추가
    dialog.append(f"System: {response}")

    return response, dialog